# Power Grid Forecasting - Demonstration Notebook
## CS604 Project 4 - Fall 2025

This notebook demonstrates the power grid forecasting system with interactive visualizations and model comparisons.

In [ ]:
# Import required libraries
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from data_loader import DataLoader
from models import ARIMAModel, LSTMModel, SimpleMovingAverage
from evaluator import Evaluator
from visualizer import Visualizer

print("Libraries imported successfully!")

## 1. Load and Explore Data

In [ ]:
# Initialize data loader
data_loader = DataLoader('../data/sample_data.csv')

# Load data (will generate synthetic data if file not found)
data = data_loader.load_data()

# Display basic info
print(f"\nData shape: {data.shape}")
print(f"Date range: {data.index.min()} to {data.index.max()}")
print(f"\nFirst few rows:")
data.head()

In [ ]:
# Display statistics
print("Statistical Summary:")
data.describe()

## 2. Visualize Time Series

In [ ]:
# Initialize visualizer
visualizer = Visualizer()

# Plot time series
visualizer.plot_time_series(data)

## 3. Analyze Seasonal Patterns

In [ ]:
# Plot seasonal patterns
visualizer.plot_seasonal_patterns(data)

## 4. Split Data into Train and Test Sets

In [ ]:
# Split data (80% train, 20% test)
train_data, test_data = data_loader.split_data(test_size=0.2)

print(f"Training set: {len(train_data)} samples")
print(f"Test set: {len(test_data)} samples")

## 5. Train and Evaluate Models

### 5.1 ARIMA Model

In [ ]:
# Train ARIMA model
arima = ARIMAModel(order=(5, 1, 2))
arima.train(train_data['power_consumption'])

# Make predictions
forecast_horizon = min(168, len(test_data))  # 1 week or less
arima_predictions = arima.predict(forecast_horizon)

print(f"\nGenerated {len(arima_predictions)} predictions")

In [ ]:
# Evaluate ARIMA
evaluator = Evaluator()
y_true = test_data['power_consumption'].values[:forecast_horizon]
arima_metrics = evaluator.evaluate_model(y_true, arima_predictions, "ARIMA")

### 5.2 LSTM Model

In [ ]:
# Train LSTM model
lstm = LSTMModel(sequence_length=24, units=50, epochs=50)
lstm.train(train_data['power_consumption'])

# Make predictions
last_sequence = train_data['power_consumption'].values[-lstm.sequence_length:]
lstm_predictions = lstm.predict(last_sequence, forecast_horizon)

print(f"\nGenerated {len(lstm_predictions)} predictions")

In [ ]:
# Evaluate LSTM
lstm_metrics = evaluator.evaluate_model(y_true, lstm_predictions, "LSTM")

### 5.3 Simple Moving Average (Baseline)

In [ ]:
# Train Moving Average model
ma = SimpleMovingAverage(window=24)
ma.train(train_data['power_consumption'])

# Make predictions
ma_predictions = ma.predict(forecast_horizon)

print(f"\nGenerated {len(ma_predictions)} predictions")

In [ ]:
# Evaluate Moving Average
ma_metrics = evaluator.evaluate_model(y_true, ma_predictions, "Moving Average")

## 6. Compare Model Performance

In [ ]:
# Compare all models
results = {
    'ARIMA': arima_metrics,
    'LSTM': lstm_metrics,
    'Moving Average': ma_metrics
}

comparison_df = evaluator.compare_models(results)

In [ ]:
# Visualize comparison
visualizer.plot_metrics_comparison(comparison_df)

## 7. Visualize Predictions

In [ ]:
# Plot predictions vs actual
test_subset = test_data['power_consumption'].iloc[:forecast_horizon]
predictions = {
    'ARIMA': arima_predictions,
    'LSTM': lstm_predictions,
    'Moving Average': ma_predictions
}

visualizer.plot_predictions(test_subset, predictions)

## 8. Residual Analysis

In [ ]:
# Analyze residuals for best performing model
# Let's use ARIMA as an example
visualizer.plot_residuals(y_true, arima_predictions, "ARIMA")

In [ ]:
# Analyze residuals for LSTM
visualizer.plot_residuals(y_true, lstm_predictions, "LSTM")

## 9. Conclusions

Based on the evaluation metrics:

1. **Best Model**: The model with the lowest RMSE and MAPE
2. **ARIMA**: Good for capturing linear trends and seasonality
3. **LSTM**: Can capture complex non-linear patterns
4. **Moving Average**: Serves as a simple baseline

The choice of model depends on:
- Computational resources available
- Required prediction accuracy
- Data characteristics (linear vs non-linear patterns)
- Interpretability requirements